<a href="https://colab.research.google.com/github/chrfranco/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/Christian%20Franco%20-%20AG2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Actividad Guiada 2 - AG2**

**Christian Franco Sánchez**

https://colab.research.google.com/drive/1XCoZ4D1ZTIBerPqAbf1VvhqBpcD9hg60

https://github.com/chrfranco/03MAIR---Algoritmos-de-Optimizacion---2019

In [0]:
# Ruta por el Río. Programación Dinámica

In [0]:
TARIFAS = [
[0,5,4,3,999,999,999],
[999,0,999,2,3,999,11],
[999,999, 0,1,999,4,10],
[999,999,999, 0,5,6,9],
[999,999, 999,999,0,999,4],
[999,999, 999,999,999,0,3],
[999,999,999,999,999,999,0]
]

In [12]:
def precios(TARIFAS):
  N=len(TARIFAS[0])
  precios = [ [9999]*N for i in[9999]*N]
  ruta = [ [""]*N for i in [""]*N]
  
  for i in range(N-1):
    for j in range(i+1,N):
      min = TARIFAS[i][j]
      ruta[i][j] = i
      
      for k in range(i,j):
        if precios[i][k] + TARIFAS[k][j] < min:
          min = precios[i][k] + TARIFAS[k][j]
          ruta[i][j] = k
          
      precios[i][j] = min
      
  return precios, ruta

precios, ruta = precios(TARIFAS)

print(precios)
print(ruta)

def calcular_ruta(ruta, desde, hasta):
  if desde == hasta:
    return desde
  else:
    return str(calcular_ruta(ruta, desde, ruta[desde][hasta])) + ' - ' + str(ruta[desde][hasta])
               
calcular_ruta(ruta, 0, 6)

[[9999, 5, 4, 3, 8, 8, 11], [9999, 9999, 999, 2, 3, 8, 7], [9999, 9999, 9999, 1, 6, 4, 7], [9999, 9999, 9999, 9999, 5, 6, 9], [9999, 9999, 9999, 9999, 9999, 999, 4], [9999, 9999, 9999, 9999, 9999, 9999, 3], [9999, 9999, 9999, 9999, 9999, 9999, 9999]]
[['', 0, 0, 0, 1, 2, 5], ['', '', 1, 1, 1, 3, 4], ['', '', '', 2, 3, 2, 5], ['', '', '', '', 3, 3, 3], ['', '', '', '', '', 4, 4], ['', '', '', '', '', '', 5], ['', '', '', '', '', '', '']]


'0 - 0 - 2 - 5'

In [0]:
# Asignación de Tareas. Técnica de Ramificación y Poda

In [0]:
import itertools
from functools import wraps
from time import time

def calcular_tiempo(f):
    @wraps(f)
    def cronometro(*args, **kwargs):
        t_inicial = time()
        salida = f(*args, **kwargs)
        t_final = time()
        print('Tiempo transcurrido (en segundos): {}'.format(t_final - t_inicial))
        return salida
    return cronometro

In [0]:
COSTES=[[11,12,18,40],
        [14,15,13,22],
        [11,17,19,23],
        [17,14,20,28]]

In [55]:
def valor(s, COSTES):
  valor = 0
  for i in range(len(s)):
    valor = valor + COSTES[s[i]][i]
  
  return valor

valor((1,0,3,2), COSTES)
                               

69

In [56]:
@calcular_tiempo
def fuerza_bruta(COSTES):
  mejor_valor = 10e10
  mejor_solucion = ''
  
  for s in list(itertools.permutations(range(len(COSTES)))):
    valor_tmp = valor(s, COSTES)
    if valor_tmp < mejor_valor:
      mejor_solucion = s
      mejor_valor = valor_tmp
      
  print("La mejor solución es:", mejor_solucion)

fuerza_bruta(COSTES)

La mejor solución es: (0, 3, 1, 2)
Tiempo transcurrido (en segundos): 0.001963376998901367


In [62]:
def CI(s, COSTES):
  valor = 0
  
  for i in range(len(s)):
    valor += COSTES[i][s[i]]
  
  # Estimaciones
  for i in range(len(COSTES)):
    if i not in s:
      valor += min([ COSTES[j][i] for j in range(len(s), len(COSTES) ) ])
  
  return valor

def crear_hijos(nodo, N):
  hijos = []
  for i in range(N):
    if i not in nodo:
      hijos.append({'s':nodo+(i,)})
      
  return hijos

crear_hijos((0,2), 4)

[{'s': (0, 2, 1)}, {'s': (0, 2, 3)}]

In [94]:
def ramificacion_y_poda(COSTES):
  dimension = len(COSTES)
  mejor_solucion = tuple(i for i in range(dimension))
  
  cotasup = valor(mejor_solucion, COSTES)
  
  nodos = []
  nodos.append({'s':(), 'ci':CI((), COSTES)})
  
  iteracion = 0
  
  while (len(nodos) > 0):
    iteracion += 1
    print('\n#', iteracion)
    nodo_prometedor = min(nodos, key=lambda x:x['ci'])
    
    # Hijos
    hijos = [{'s':x['s'], 'ci':CI(x['s'], COSTES)} for x in crear_hijos(nodo_prometedor['s'], dimension)]
    
    nodo_final = [x for x in hijos if len(x['s']) == dimension]
    if len(nodo_final) > 0:
      if nodo_final[0]['ci'] < cotasup:
        cotasup = nodo_final[0]['ci']
        mejor_solucion = nodo_final[0]
        
    # Poda
    hijos = [x for x in hijos if x['ci'] < cotasup]
    
    # Eliminamos el nodo expandido
    nodos = [x for x in nodos if x['s'] != nodo_prometedor['s']]
    
    # Añadimos los hijos
    nodos.extend(hijos)
    
    print(nodos)
    
  print("La mejor solución es:", mejor_solucion)
  
ramificacion_y_poda(COSTES)


# 1
[{'s': (0,), 'ci': 60}, {'s': (1,), 'ci': 58}, {'s': (2,), 'ci': 65}]

# 2
[{'s': (0,), 'ci': 60}, {'s': (2,), 'ci': 65}, {'s': (1, 0), 'ci': 68}, {'s': (1, 2), 'ci': 59}, {'s': (1, 3), 'ci': 64}]

# 3
[{'s': (0,), 'ci': 60}, {'s': (2,), 'ci': 65}, {'s': (1, 0), 'ci': 68}, {'s': (1, 3), 'ci': 64}, {'s': (1, 2, 0), 'ci': 64}, {'s': (1, 2, 3), 'ci': 65}]

# 4
[{'s': (2,), 'ci': 65}, {'s': (1, 0), 'ci': 68}, {'s': (1, 3), 'ci': 64}, {'s': (1, 2, 0), 'ci': 64}, {'s': (1, 2, 3), 'ci': 65}, {'s': (0, 1), 'ci': 68}, {'s': (0, 2), 'ci': 61}, {'s': (0, 3), 'ci': 66}]

# 5
[{'s': (2,), 'ci': 65}, {'s': (1, 0), 'ci': 68}, {'s': (1, 3), 'ci': 64}, {'s': (1, 2, 0), 'ci': 64}, {'s': (1, 2, 3), 'ci': 65}, {'s': (0, 1), 'ci': 68}, {'s': (0, 3), 'ci': 66}, {'s': (0, 2, 1), 'ci': 69}, {'s': (0, 2, 3), 'ci': 61}]

# 6
[{'s': (2,), 'ci': 65}, {'s': (1, 0), 'ci': 68}, {'s': (1, 3), 'ci': 64}, {'s': (1, 2, 0), 'ci': 64}, {'s': (1, 2, 3), 'ci': 65}, {'s': (0, 1), 'ci': 68}, {'s': (0, 3), 'ci': 66}, {'s'

In [25]:
list(itertools.permutations(range(len(COSTES))))

[(0, 1, 2, 3),
 (0, 1, 3, 2),
 (0, 2, 1, 3),
 (0, 2, 3, 1),
 (0, 3, 1, 2),
 (0, 3, 2, 1),
 (1, 0, 2, 3),
 (1, 0, 3, 2),
 (1, 2, 0, 3),
 (1, 2, 3, 0),
 (1, 3, 0, 2),
 (1, 3, 2, 0),
 (2, 0, 1, 3),
 (2, 0, 3, 1),
 (2, 1, 0, 3),
 (2, 1, 3, 0),
 (2, 3, 0, 1),
 (2, 3, 1, 0),
 (3, 0, 1, 2),
 (3, 0, 2, 1),
 (3, 1, 0, 2),
 (3, 1, 2, 0),
 (3, 2, 0, 1),
 (3, 2, 1, 0)]